# Black Box Recording Demo

This notebook demonstrates the **BlackBoxRecorder** - an aviation-style flight recorder for AI agent workflows.

## What You'll Learn
1. Recording task plans with steps and dependencies
2. Tracking collaborating agents
3. Logging parameter substitutions
4. Capturing execution traces
5. Exporting and replaying recordings


In [1]:
# Setup imports
import sys
from pathlib import Path
from datetime import datetime, UTC

# Add lesson-17 to path
sys.path.insert(0, str(Path.cwd().parent))

from backend.explainability.black_box import (
    BlackBoxRecorder,
    TaskPlan,
    PlanStep,
    AgentInfo,
    ExecutionTrace,
    TraceEvent,
    EventType,
)

print("Imports successful!")


Imports successful!


In [2]:
# Load sample workflow data from the data directory
import json

data_path = Path.cwd().parent / "data" / "workflows" / "invoice_processing_trace.json"
with open(data_path) as f:
    workflow_data = json.load(f)

print(f"Loaded workflow: {workflow_data['workflow_id']}")
print(f"Workflow type: {workflow_data['workflow_type']}")
print(f"Outcome: {workflow_data['outcome']['status']}")
print(f"\nThis workflow demonstrates a cascade failure scenario:")
print(f"  - Root cause: {workflow_data['outcome']['root_cause']}")


Loaded workflow: invoice-processing-001
Workflow type: invoice_processing
Outcome: failed

This workflow demonstrates a cascade failure scenario:
  - Root cause: Parameter substitution (confidence_threshold: 0.8 → 0.95) caused empty validation results


## 1. Initialize the Black Box Recorder

Each recorder is associated with a workflow and stores data in a specified directory.


In [3]:
# Create a recorder for an invoice processing workflow
storage_path = Path.cwd().parent / "cache"
recorder = BlackBoxRecorder(
    workflow_id="invoice-processing-001",
    storage_path=storage_path
)

print(f"Recorder initialized for workflow: {recorder.workflow_id}")
print(f"Storage path: {recorder._recordings_path}")


Recorder initialized for workflow: invoice-processing-001
Storage path: /Users/rajnishkhatri/Documents/recipe-chatbot/lesson-17/cache/black_box_recordings/invoice-processing-001


## 2. Record a Task Plan

Task plans capture the intended execution steps, their dependencies, and rollback points.


In [4]:
# Define a task plan for invoice extraction
plan = TaskPlan(
    plan_id="plan-inv-001",
    task_id="task-extract-invoice",
    steps=[
        PlanStep(
            step_id="step-1-extract",
            description="Extract vendor and amount from invoice",
            agent_id="invoice-extractor",
            expected_inputs=["invoice_text"],
            expected_outputs=["vendor_name", "amount"],
            timeout_seconds=60,
            is_critical=True,
            order=1,
        ),
        PlanStep(
            step_id="step-2-validate",
            description="Validate extracted data against database",
            agent_id="data-validator",
            expected_inputs=["vendor_name", "amount"],
            expected_outputs=["validation_result", "vendor_id"],
            timeout_seconds=30,
            is_critical=True,
            order=2,
        ),
    ],
    dependencies={"step-2-validate": ["step-1-extract"]},
    rollback_points=["step-1-extract"],
)

# Record the plan
recorder.record_task_plan("task-extract-invoice", plan)
print(f"Recorded task plan with {len(plan.steps)} steps")


Recorded task plan with 2 steps


## 3. Record Collaborating Agents

The black box tracks which agents participated in the workflow. Each agent's role, capabilities, and join time are recorded.

In [5]:
# Create AgentInfo objects from the workflow data
from datetime import datetime

collaborators = []
for collab_data in workflow_data["collaborators"]:
    agent = AgentInfo(
        agent_id=collab_data["agent_id"],
        agent_name=collab_data["agent_name"],
        role=collab_data["role"],
        joined_at=datetime.fromisoformat(collab_data["joined_at"]),
        capabilities=["extraction", "ocr"] if "extractor" in collab_data["agent_id"] else ["validation", "database_lookup"]
    )
    collaborators.append(agent)
    print(f"Agent: {agent.agent_name}")
    print(f"  - ID: {agent.agent_id}")
    print(f"  - Role: {agent.role}")
    print(f"  - Capabilities: {agent.capabilities}")
    print()

# Record the collaborators
recorder.record_collaborators("task-extract-invoice", collaborators)
print(f"✓ Recorded {len(collaborators)} collaborating agents")

Agent: Invoice Extractor
  - ID: invoice-extractor-v2
  - Role: extraction
  - Capabilities: ['extraction', 'ocr']

Agent: Amount Validator
  - ID: invoice-validator-v1
  - Role: validation
  - Capabilities: ['validation', 'database_lookup']

✓ Recorded 2 collaborating agents


## 4. Record Parameter Substitutions

Parameter substitutions log changes to configuration values with before/after states and justification.
This is critical for **root cause analysis** when changes cause cascade failures.


In [6]:
# Record parameter substitutions from the workflow data
# This is the change that caused the cascade failure!
param_sub = workflow_data["parameter_substitutions"][0]

recorder.record_parameter_substitution(
    task_id="task-extract-invoice",
    param=param_sub["parameter_name"],
    old_val=param_sub["old_value"],
    new_val=param_sub["new_value"],
    reason=param_sub["justification"],
    agent_id=param_sub["changed_by"]
)

print("Parameter Substitution Recorded:")
print(f"  Parameter: {param_sub['parameter_name']}")
print(f"  Old Value: {param_sub['old_value']}")
print(f"  New Value: {param_sub['new_value']}")
print(f"  Reason: {param_sub['justification']}")
print(f"  Changed By: {param_sub['changed_by']}")
print()
print("⚠️  This parameter change (0.8 → 0.95) is the ROOT CAUSE of the cascade failure!")


Parameter Substitution Recorded:
  Parameter: confidence_threshold
  Old Value: 0.8
  New Value: 0.95
  Reason: Reduce false positives per compliance team request
  Changed By: invoice-extractor-v2

⚠️  This parameter change (0.8 → 0.95) is the ROOT CAUSE of the cascade failure!


## 5. Record Execution Trace

The execution trace captures every event during workflow execution, including:
- **STEP_START / STEP_END**: Step lifecycle
- **DECISION**: Choices made by agents
- **CHECKPOINT**: State snapshots for recovery
- **ERROR**: Failures and exceptions
- **PARAMETER_CHANGE**: Runtime configuration changes
- **COLLABORATOR_JOIN / COLLABORATOR_LEAVE**: Agent lifecycle


In [7]:
# Build TraceEvent objects from the workflow data
trace_data = workflow_data["execution_trace"]

# Map string event types to EventType enum
event_type_map = {
    "step_start": EventType.STEP_START,
    "step_end": EventType.STEP_END,
    "decision": EventType.DECISION,
    "error": EventType.ERROR,
    "checkpoint": EventType.CHECKPOINT,
    "parameter_change": EventType.PARAMETER_CHANGE,
    "collaborator_join": EventType.COLLABORATOR_JOIN,
    "collaborator_leave": EventType.COLLABORATOR_LEAVE,
}

trace_events = []
for evt in trace_data["events"]:
    trace_event = TraceEvent(
        event_id=evt["event_id"],
        timestamp=datetime.fromisoformat(evt["timestamp"]),
        event_type=event_type_map[evt["event_type"]],
        agent_id=evt.get("agent_id"),
        step_id=evt.get("step_id"),
        duration_ms=evt.get("duration_ms"),
        metadata=evt.get("metadata", {})
    )
    trace_events.append(trace_event)

# Create the ExecutionTrace
execution_trace = ExecutionTrace(
    trace_id=trace_data["trace_id"],
    task_id=trace_data["task_id"],
    start_time=datetime.fromisoformat(trace_data["start_time"]),
    end_time=datetime.fromisoformat(trace_data["end_time"]),
    events=trace_events,
    final_outcome=workflow_data["outcome"]["status"],
    error_chain=[workflow_data["outcome"]["reason"], workflow_data["outcome"]["root_cause"]]
)

# Record the trace
recorder.record_execution_trace("task-extract-invoice", execution_trace)

print(f"Recorded execution trace with {len(trace_events)} events:")
print()
for evt in trace_events:
    icon = {"step_start": "▶", "step_end": "■", "decision": "◆", "error": "✗", 
            "checkpoint": "💾", "parameter_change": "⚙", "collaborator_join": "→", 
            "collaborator_leave": "←"}.get(evt.event_type.value, "•")
    print(f"  {icon} {evt.event_type.value:20} | {evt.step_id or 'N/A':15} | {evt.agent_id or 'N/A'}")


Recorded execution trace with 12 events:

  ▶ step_start           | extract_vendor  | invoice-extractor-v2
  → collaborator_join    | extract_vendor  | invoice-extractor-v2
  ◆ decision             | extract_vendor  | invoice-extractor-v2
  ⚙ parameter_change     | extract_vendor  | invoice-extractor-v2
  💾 checkpoint           | extract_vendor  | invoice-extractor-v2
  ■ step_end             | extract_vendor  | invoice-extractor-v2
  ← collaborator_leave   | extract_vendor  | invoice-extractor-v2
  ▶ step_start           | validate_amount | invoice-validator-v1
  → collaborator_join    | validate_amount | invoice-validator-v1
  ✗ error                | validate_amount | invoice-validator-v1
  ■ step_end             | validate_amount | invoice-validator-v1
  ← collaborator_leave   | validate_amount | invoice-validator-v1


## 5.5. Root Cause Analysis

Now let's programmatically analyze the execution trace to identify the root cause of the failure by correlating events chronologically.

In [8]:
# Perform root cause analysis by analyzing execution trace events
def analyze_root_cause(trace: ExecutionTrace) -> dict:
    """Analyze execution trace to identify root cause of failures.
    
    Returns a dictionary with:
    - root_cause: Identified root cause
    - evidence_chain: Chronological sequence of events leading to failure
    - parameter_changes: Parameter changes that occurred
    - errors: Errors that occurred
    - correlation: How parameter changes correlate with errors
    """
    # Sort events chronologically
    sorted_events = sorted(trace.events, key=lambda e: e.timestamp)
    
    # Extract key events
    parameter_changes = []
    errors = []
    step_outputs = {}  # Track step outputs (confidence, etc.)
    
    for event in sorted_events:
        if event.event_type == EventType.PARAMETER_CHANGE:
            param_info = {
                "timestamp": event.timestamp,
                "step_id": event.step_id,
                "agent_id": event.agent_id,
                "parameter": event.metadata.get("parameter"),
                "old_value": event.metadata.get("old_value"),
                "new_value": event.metadata.get("new_value"),
            }
            parameter_changes.append(param_info)
        
        elif event.event_type == EventType.ERROR:
            error_info = {
                "timestamp": event.timestamp,
                "step_id": event.step_id,
                "agent_id": event.agent_id,
                "error_message": event.metadata.get("error_message", ""),
                "error_type": event.metadata.get("error_type", ""),
            }
            errors.append(error_info)
        
        elif event.event_type == EventType.STEP_END:
            # Capture step outputs that might be relevant
            if event.metadata.get("confidence"):
                step_outputs[event.step_id] = {
                    "confidence": event.metadata.get("confidence"),
                    "timestamp": event.timestamp,
                }
    
    # Correlate parameter changes with errors
    correlations = []
    for param_change in parameter_changes:
        # Find errors that occurred after this parameter change
        subsequent_errors = [
            err for err in errors 
            if err["timestamp"] > param_change["timestamp"]
        ]
        
        # Check if error messages reference the changed parameter
        for error in subsequent_errors:
            error_msg = error["error_message"].lower()
            param_name = param_change["parameter"].lower() if param_change["parameter"] else ""
            
            # Check if error mentions the parameter or its value
            mentions_param = (
                param_name in error_msg or 
                str(param_change["new_value"]) in error_msg
            )
            
            if mentions_param or len(subsequent_errors) == 1:
                # Check if step outputs are affected
                affected_output = None
                for step_id, output in step_outputs.items():
                    if output["timestamp"] > param_change["timestamp"]:
                        # Check if output value conflicts with new parameter
                        if param_change["parameter"] == "confidence_threshold":
                            if "confidence" in output:
                                if output["confidence"] < param_change["new_value"]:
                                    affected_output = {
                                        "step_id": step_id,
                                        "confidence": output["confidence"],
                                        "threshold": param_change["new_value"],
                                    }
                
                correlation = {
                    "parameter_change": param_change,
                    "error": error,
                    "time_delta_seconds": (error["timestamp"] - param_change["timestamp"]).total_seconds(),
                    "affected_output": affected_output,
                    "confidence": "high" if mentions_param else "medium",
                }
                correlations.append(correlation)
    
    # Determine root cause
    root_cause = None
    if correlations:
        # Use the first (earliest) correlation as root cause
        best_correlation = correlations[0]
        param = best_correlation["parameter_change"]
        error = best_correlation["error"]
        affected = best_correlation.get("affected_output")
        
        if affected:
            root_cause = (
                f"Parameter substitution ({param['parameter']}: {param['old_value']} → {param['new_value']}) "
                f"caused {error['error_type']} in step '{error['step_id']}'. "
                f"Step output ({affected['step_id']}: confidence={affected['confidence']}) "
                f"was below new threshold ({affected['threshold']})."
            )
        else:
            root_cause = (
                f"Parameter substitution ({param['parameter']}: {param['old_value']} → {param['new_value']}) "
                f"caused {error['error_type']} in step '{error['step_id']}': {error['error_message']}"
            )
    elif parameter_changes and errors:
        # Fallback: parameter change before error
        root_cause = (
            f"Parameter change ({parameter_changes[0]['parameter']}: "
            f"{parameter_changes[0]['old_value']} → {parameter_changes[0]['new_value']}) "
            f"preceded error in step '{errors[0]['step_id']}'"
        )
    elif errors:
        root_cause = f"Error in step '{errors[0]['step_id']}': {errors[0]['error_message']}"
    else:
        root_cause = "No root cause identified from trace analysis"
    
    return {
        "root_cause": root_cause,
        "parameter_changes": parameter_changes,
        "errors": errors,
        "correlations": correlations,
        "step_outputs": step_outputs,
    }


# Perform the analysis
# Use the execution_trace variable created in the previous cell (section 5)
# If that's not available, try to get it from the recorder
try:
    # First try to use the execution_trace variable from the previous cell
    trace = execution_trace
except NameError:
    # Fallback: get from recorder
    trace = recorder.get_execution_trace("task-extract-invoice")
    if trace is None:
        print("❌ Error: Execution trace not found.")
        print("   Please run the 'Record Execution Trace' cell (section 5) first.")
        raise ValueError("Execution trace not available")

if not trace.events:
    print("❌ Error: Execution trace has no events.")
    raise ValueError("Cannot perform root cause analysis without execution trace events")

analysis = analyze_root_cause(trace)

print("=" * 70)
print("ROOT CAUSE ANALYSIS")
print("=" * 70)
print()

print("📊 IDENTIFIED ROOT CAUSE:")
print(f"   {analysis['root_cause']}")
print()

if analysis['parameter_changes']:
    print("⚙️  PARAMETER CHANGES DETECTED:")
    for i, param in enumerate(analysis['parameter_changes'], 1):
        timestamp_str = param['timestamp'].strftime("%H:%M:%S")
        print(f"   {i}. [{timestamp_str}] {param['parameter']}: {param['old_value']} → {param['new_value']}")
        print(f"      Step: {param['step_id']} | Agent: {param['agent_id']}")
    print()

if analysis['errors']:
    print("✗ ERRORS DETECTED:")
    for i, error in enumerate(analysis['errors'], 1):
        timestamp_str = error['timestamp'].strftime("%H:%M:%S")
        print(f"   {i}. [{timestamp_str}] {error['error_type']} in step '{error['step_id']}'")
        print(f"      Message: {error['error_message']}")
        print(f"      Agent: {error['agent_id']}")
    print()

if analysis['correlations']:
    print("🔗 CAUSAL CORRELATIONS:")
    for i, corr in enumerate(analysis['correlations'], 1):
        param = corr['parameter_change']
        error = corr['error']
        print(f"   {i}. Parameter change → Error (Δ{corr['time_delta_seconds']:.1f}s)")
        print(f"      Parameter: {param['parameter']} = {param['new_value']}")
        error_msg_short = error['error_message'][:60] + "..." if len(error['error_message']) > 60 else error['error_message']
        print(f"      Error: {error['error_type']} - {error_msg_short}")
        
        if corr.get('affected_output'):
            affected = corr['affected_output']
            print(f"      ⚠️  Output conflict: confidence {affected['confidence']} < threshold {affected['threshold']}")
        print(f"      Confidence: {corr['confidence']}")
        print()
else:
    print("ℹ️  No direct correlations found between parameter changes and errors.")
    print()

ROOT CAUSE ANALYSIS

📊 IDENTIFIED ROOT CAUSE:
   Parameter substitution (confidence_threshold: 0.8 → 0.95) caused ValidationError in step 'validate_amount'. Step output (extract_vendor: confidence=0.92) was below new threshold (0.95).

⚙️  PARAMETER CHANGES DETECTED:
   1. [14:00:10] confidence_threshold: 0.8 → 0.95
      Step: extract_vendor | Agent: invoice-extractor-v2

✗ ERRORS DETECTED:
   1. [14:00:15] ValidationError in step 'validate_amount'
      Message: Confidence threshold too high (0.95) - no valid results
      Agent: invoice-validator-v1

🔗 CAUSAL CORRELATIONS:
   1. Parameter change → Error (Δ5.0s)
      Parameter: confidence_threshold = 0.95
      Error: ValidationError - Confidence threshold too high (0.95) - no valid results
      ⚠️  Output conflict: confidence 0.92 < threshold 0.95
      Confidence: high



## 6. Add Individual Trace Events

You can also add events incrementally as they occur, rather than recording a complete trace at once.
This is useful for real-time monitoring and debugging.


In [9]:
# Add a ROLLBACK event to show the system attempting recovery
rollback_event = TraceEvent(
    event_id="evt-013",
    timestamp=datetime.now(UTC),
    event_type=EventType.ROLLBACK,
    agent_id="workflow-orchestrator",
    step_id="extract_vendor",
    metadata={
        "rollback_reason": "Cascade failure detected",
        "rollback_to": "extract_vendor",
        "steps_rolled_back": ["validate_amount"],
        "recovery_action": "Retry with original parameters"
    }
)

recorder.add_trace_event("task-extract-invoice", rollback_event)

print("Added ROLLBACK event to trace:")
print(f"  Event ID: {rollback_event.event_id}")
print(f"  Type: {rollback_event.event_type.value}")
print(f"  Rollback to: {rollback_event.metadata['rollback_to']}")
print(f"  Reason: {rollback_event.metadata['rollback_reason']}")

# Verify the event was added
trace = recorder.get_execution_trace("task-extract-invoice")
print(f"\n✓ Trace now has {len(trace.events)} events (was 12)")


Added ROLLBACK event to trace:
  Event ID: evt-013
  Type: rollback
  Rollback to: extract_vendor
  Reason: Cascade failure detected

✓ Trace now has 13 events (was 12)


## 7. Export Black Box

Export all recordings for a task to a single JSON file for:
- **Compliance auditing**: Provide complete audit trails
- **Post-incident analysis**: Share with investigation teams
- **Archival**: Long-term storage of workflow history


In [10]:
# Export the complete black box to a JSON file
export_path = Path.cwd().parent / "cache" / "exports" / "task-extract-invoice-blackbox.json"

recorder.export_black_box("task-extract-invoice", export_path)

print(f"✓ Exported black box to: {export_path}")
print()

# Show the structure of the exported file
with open(export_path) as f:
    export_data = json.load(f)

print("Export structure:")
print(f"  - workflow_id: {export_data['workflow_id']}")
print(f"  - task_id: {export_data['task_id']}")
print(f"  - exported_at: {export_data['exported_at']}")
print(f"  - task_plan: {len(export_data['task_plan']['steps'])} steps")
print(f"  - collaborators: {len(export_data['collaborators'])} agents")
print(f"  - parameter_substitutions: {len(export_data['parameter_substitutions'])} changes")
print(f"  - execution_trace: {len(export_data['execution_trace']['events'])} events")
print(f"  - all_events: {len(export_data['all_events'])} total recorded events")


✓ Exported black box to: /Users/rajnishkhatri/Documents/recipe-chatbot/lesson-17/cache/exports/task-extract-invoice-blackbox.json

Export structure:
  - workflow_id: invoice-processing-001
  - task_id: task-extract-invoice
  - exported_at: 2025-11-29T13:22:40.876932+00:00
  - task_plan: 2 steps
  - collaborators: 2 agents
  - parameter_substitutions: 1 changes
  - execution_trace: 13 events
  - all_events: 17 total recorded events


## 8. Replay Events

The `replay()` method returns an iterator over all recorded events in chronological order.
This is essential for debugging and post-mortem analysis.


In [11]:
# Replay all recorded events in chronological order
print("Replaying events in chronological order:")
print("=" * 70)
print()

event_count = 0
for recorded_event in recorder.replay("task-extract-invoice"):
    event_count += 1
    # Show first 10 events as a sample
    if event_count <= 10:
        timestamp_str = recorded_event.timestamp.strftime("%H:%M:%S.%f")[:-3]
        print(f"[{timestamp_str}] {recorded_event.event_type}")
        
        # Show key metadata for certain event types
        if recorded_event.event_type == "parameter_substitution":
            print(f"           └─ {recorded_event.data.get('param_name')}: "
                  f"{recorded_event.data.get('old_value')} → {recorded_event.data.get('new_value')}")
        elif recorded_event.event_type == "task_plan":
            print(f"           └─ Plan: {recorded_event.data.get('plan_id')} "
                  f"({len(recorded_event.data.get('steps', []))} steps)")
    elif event_count == 11:
        print("... (truncated)")

print()
print(f"Total events replayed: {event_count}")


Replaying events in chronological order:

[14:00:00.000] collaborator_join
[14:00:00.000] trace_step_start
[14:00:00.000] trace_collaborator_join
[14:00:05.000] trace_decision
[14:00:10.000] trace_parameter_change
[14:00:11.000] trace_checkpoint
[14:00:12.000] collaborator_join
[14:00:12.000] trace_step_end
[14:00:12.000] trace_collaborator_leave
[14:00:12.000] trace_step_start
... (truncated)

Total events replayed: 17


## 9. Utility Methods

The BlackBoxRecorder provides getter methods and a hash utility for integrity verification.


In [12]:
# Getter methods - retrieve recorded data
print("=== Getter Methods ===")
print()

# Get task plan
retrieved_plan = recorder.get_task_plan("task-extract-invoice")
if retrieved_plan:
    print(f"get_task_plan(): Retrieved plan '{retrieved_plan.plan_id}' with {len(retrieved_plan.steps)} steps")
else:
    print("get_task_plan(): No plan found")

# Get collaborators
retrieved_collabs = recorder.get_collaborators("task-extract-invoice")
print(f"get_collaborators(): Found {len(retrieved_collabs)} agents: {[a.agent_name for a in retrieved_collabs]}")

# Get execution trace
retrieved_trace = recorder.get_execution_trace("task-extract-invoice")
if retrieved_trace:
    print(f"get_execution_trace(): Retrieved trace '{retrieved_trace.trace_id}' with {len(retrieved_trace.events)} events")
    print(f"                       Final outcome: {retrieved_trace.final_outcome}")
else:
    print("get_execution_trace(): No trace found")

print()
print("=== Hash Utility for Integrity Verification ===")
print()

# Compute hash - useful for verifying data integrity
sample_input = {"vendor_name": "Acme Corp", "amount": 4523.50}
sample_output = {"validation_result": "failed", "error": "threshold_exceeded"}

input_hash = BlackBoxRecorder.compute_hash(sample_input)
output_hash = BlackBoxRecorder.compute_hash(sample_output)

print(f"Sample input:  {sample_input}")
print(f"Input hash:    {input_hash[:32]}...")
print()
print(f"Sample output: {sample_output}")
print(f"Output hash:   {output_hash[:32]}...")
print()
print("These hashes can be stored in TraceEvent.input_hash and TraceEvent.output_hash")
print("for tamper-evident audit trails.")


=== Getter Methods ===

get_task_plan(): Retrieved plan 'plan-inv-001' with 2 steps
get_collaborators(): Found 2 agents: ['Invoice Extractor', 'Amount Validator']
get_execution_trace(): Retrieved trace 'trace-invoice-processing-001' with 13 events
                       Final outcome: failed

=== Hash Utility for Integrity Verification ===

Sample input:  {'vendor_name': 'Acme Corp', 'amount': 4523.5}
Input hash:    36d3c9b1460a710cd4bb3a734eeae339...

Sample output: {'validation_result': 'failed', 'error': 'threshold_exceeded'}
Output hash:   eb4256d40ac18c2bd867dd9737fd4006...

These hashes can be stored in TraceEvent.input_hash and TraceEvent.output_hash
for tamper-evident audit trails.


## 10. Summary

### What We Covered

| Method | Purpose |
|--------|---------|
| `record_task_plan()` | Persist task plans with steps, dependencies, rollback points |
| `record_collaborators()` | Track which agents participated and their roles |
| `record_parameter_substitution()` | Log configuration changes with before/after values |
| `record_execution_trace()` | Store complete execution history with all events |
| `add_trace_event()` | Append individual events to existing traces |
| `export_black_box()` | Export all recordings to a single JSON file |
| `replay()` | Iterator for replaying events in chronological order |
| `get_task_plan()` / `get_collaborators()` / `get_execution_trace()` | Retrieve recorded data |
| `compute_hash()` | SHA256 hash for integrity verification |

### Key Use Cases

1. **Post-Incident Analysis**: Trace cascade failures back to root causes (like the parameter change in this demo)
2. **Compliance Auditing**: Provide complete, tamper-evident audit trails
3. **Debugging**: Replay events to understand exactly what happened
4. **Workflow Optimization**: Analyze execution patterns and bottlenecks

### Event Types Demonstrated

- `STEP_START` / `STEP_END` - Step lifecycle
- `DECISION` - Agent choices with rationale
- `CHECKPOINT` - State snapshots for recovery
- `ERROR` - Failures and exceptions
- `PARAMETER_CHANGE` - Runtime configuration changes
- `COLLABORATOR_JOIN` / `COLLABORATOR_LEAVE` - Agent lifecycle
- `ROLLBACK` - Recovery attempts
